# Load Util File with TARE Model Functions

In [1]:
import os

# Measure Package 0: Baseline
menu_mp = 0
input_mp = 'baseline'

# Get the current working directory of the project
project_root = os.path.abspath(os.getcwd())
print(f"Project root directory: {project_root}")

# Relative path to the file from the project root
relative_path = r"tare_model_functions_v2.2.ipynb"

# Construct the absolute path to the file
file_path = os.path.join(project_root, relative_path)
print(f"File path: {file_path}")

# Run the notebook and import variables
if os.path.exists(relative_path):
    get_ipython().run_line_magic('run', f'-i "{relative_path}"')
    print("Loaded All TARE Model Functions")
else:
    print(f"File not found: {relative_path}")

Project root directory: c:\Users\14128\Research\cmu-tare-model
File path: c:\Users\14128\Research\cmu-tare-model\tare_model_functions_v2.2.ipynb
Loaded All TARE Model Functions


In [2]:
# Storing Result Outputs in output_results folder
relative_path = r"output_results"
output_folder_path = os.path.join(project_root, relative_path)
print(f"Result outputs will be exported here: {output_folder_path}")

Result outputs will be exported here: c:\Users\14128\Research\cmu-tare-model\output_results


# Baseline: 

## Simulate Residential Energy Consumption
- Filter EUSS Data: Only occupied units and Single Family Homes



In [3]:
# The ``inline`` flag will use the appropriate backend to make figures appear inline in the notebook.  
%matplotlib inline

import pandas as pd
import numpy as np

# `plt` is an alias for the `matplotlib.pyplot` module
import matplotlib.pyplot as plt

# import seaborn library (wrapper of matplotlib)
import seaborn as sns
sns.set(style="darkgrid")

# For regex, import re
import re

from datetime import datetime

# Get the current datetime
# Start the timer
start_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [4]:
print("""
-------------------------------------------------------------------------------------------------------
Welcome to the Trade-off Analysis of residential Retrofits for energy Equity Tool (TARE Model)
Let's start by reading the data from the NREL EUSS Database.

Make sure that the zipped folders stay organized as they are once unzipped.
If changes are made to the file path, then the program will not run properly.
-------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------
BASELINE (Measure Package 0)
-------------------------------------------------------------------------------------------------------
""")

# Measure Package 0: Baseline
menu_mp = 0
input_mp = 'baseline'

filename = "baseline_metadata_and_annual_results.csv"
relative_path = os.path.join(r"euss_data\resstock_amy2018_release_1.1\state", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

print("""
-------------------------------------------------------------------------------------------------------
Data Filters: Only occupied units and Single Family Homes
-------------------------------------------------------------------------------------------------------
""")

# Fix DtypeWarning error in columns:
# 'in.neighbors', 'in.geometry_stories_low_rise', 'in.iso_rto_region', 'in.pv_orientation', 'in.pv_system_size'
columns_to_string = {11: str, 61: str, 121: str, 103: str, 128: str, 129: str}
df_euss_am_baseline = pd.read_csv(file_path, dtype=columns_to_string)
occupancy_filter = df_euss_am_baseline['in.vacancy_status'] == 'Occupied'
df_euss_am_baseline = df_euss_am_baseline.loc[occupancy_filter]

# Filter for single family home building type
house_type_list = ['Single-Family Attached', 'Single-Family Detached']
house_type_filter = df_euss_am_baseline['in.geometry_building_type_recs'].isin(house_type_list)
df_euss_am_baseline = df_euss_am_baseline.loc[house_type_filter]
# df_euss_am_baseline


-------------------------------------------------------------------------------------------------------
Welcome to the Trade-off Analysis of residential Retrofits for energy Equity Tool (TARE Model)
Let's start by reading the data from the NREL EUSS Database.

Make sure that the zipped folders stay organized as they are once unzipped.
If changes are made to the file path, then the program will not run properly.
-------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------
BASELINE (Measure Package 0)
-------------------------------------------------------------------------------------------------------

Retrieved data for filename: baseline_metadata_and_annual_results.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\euss_data\resstock_amy2018_release_1.1\state\baseline_metadata_and_annual_results.csv

------------------------

In [5]:
# # Make a copy of the dataframe
# df_euss_am_baseline = df_euss_am_baseline.copy()

# Choose between national or sub-national level analysis
menu_state = get_menu_choice(menu_prompt, {'N', 'Y'})   # This code is only run in baseline

# National Level 
if menu_state == 'N':
    print("You chose to analyze all of the United States.")
    input_state = 'National'

# Filter down to state or city
else:
    input_state = get_state_choice(df_euss_am_baseline)
    print(f"You chose to filter for: {input_state}")
    state_filter = df_euss_am_baseline['in.state'].eq(input_state)
    df_euss_am_baseline = df_euss_am_baseline.loc[state_filter]

    print(city_prompt)
    print(df_euss_am_baseline['in.city'].value_counts())

    menu_city = get_menu_choice(city_menu_prompt, {'N', 'Y'})

    # Filter for the entire selected state
    if menu_city == 'N':
        print(f"You chose to analyze all of state: {input_state}")
        
    # Filter to a city within the selected state
    else:
        input_cityFilter = get_city_choice(df_euss_am_baseline, input_state)
        print(f"You chose to filter for: {input_state}, {input_cityFilter}")
        city_filter = df_euss_am_baseline['in.city'].eq(f"{input_state}, {input_cityFilter}")
        df_euss_am_baseline = df_euss_am_baseline.loc[city_filter]

# Display the filtered dataframe
df_euss_am_baseline

You chose to filter for: PA

To accurately characterize load profile, it is recommended to select subsets of data with >= 1000 models (~240,000 representative dwelling units).

The following cities (number of models also shown) are available for this state:

in.city
Not in a census Place      7035
In another census Place    5851
PA, Philadelphia           1631
PA, Pittsburgh              329
PA, Erie                    105
PA, Allentown               104
PA, Reading                  92
PA, Bethlehem                85
PA, Scranton                 80
PA, Harrisburg               63
PA, Lancaster                58
PA, Levittown                58
PA, Altoona                  56
PA, York                     56
PA, Wilkes-Barre             48
Name: count, dtype: int64
You chose to filter for: PA, Pittsburgh


,bldg_id,upgrade,weight,applicability,in.sqft,in.ahs_region,in.ashrae_iecc_climate_zone_2004,in.ashrae_iecc_climate_zone_2004_2_a_split,in.bathroom_spot_vent_hour,in.bedrooms,...,out.emissions.natural_gas.lrmer_low_re_cost_25_2025_start.co2e_kg,out.emissions.propane.lrmer_low_re_cost_25_2025_start.co2e_kg,out.emissions.electricity.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.fuel_oil.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.natural_gas.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.propane.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_95_decarb_by_2035_15_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_low_re_cost_15_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_low_re_cost_25_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_mid_case_15_2025_start.co2e_kg
410884,1081,0,242.131013,True,1690.0,Non-CBSA Middle Atlantic,5A,5A,Hour17,3,...,13221.094491,0.0,4119.485081,0.0,13221.094491,0.0,14937.340601,16922.278503,16688.533282,17340.579572
411011,4075,0,242.131013,True,2152.0,Non-CBSA Middle Atlantic,5A,5A,Hour9,4,...,7522.693379,0.0,3693.337771,0.0,7522.693379,0.0,9041.994672,10856.078389,10587.106731,11216.031150
411058,5187,0,242.131013,True,1220.0,Non-CBSA Middle Atlantic,5A,5A,Hour4,3,...,6830.000677,0.0,2902.206907,0.0,6830.000677,0.0,8027.212378,9460.373305,9275.214632,9732.207584
411079,5617,0,242.131013,True,1690.0,Non-CBSA Middle Atlantic,5A,5A,Hour20,2,...,8543.069827,0.0,2258.450018,0.0,8543.069827,0.0,9477.052350,10591.396808,10416.016225,10801.519845
411105,6051,0,242.131013,True,1220.0,Non-CBSA Middle Atlantic,5A,5A,Hour23,2,...,8094.655667,0.0,3136.650659,0.0,8094.655667,0.0,9386.093472,10947.632380,10781.152187,11231.306326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433460,539204,0,242.131013,True,2176.0,Non-CBSA Middle Atlantic,5A,5A,Hour0,4,...,11885.289002,0.0,5412.661506,0.0,11885.289002,0.0,14108.870013,16828.413912,16417.984031,17297.950507
433547,541686,0,242.131013,True,1690.0,Non-CBSA Middle Atlantic,5A,5A,Hour4,3,...,6844.549199,0.0,3929.994147,0.0,6844.549199,0.0,8447.084238,10392.046590,10074.405832,10774.543346
433583,542571,0,242.131013,True,1220.0,Non-CBSA Middle Atlantic,5A,5A,Hour22,3,...,2646.574040,0.0,2941.717524,0.0,2646.574040,0.0,3830.248278,5289.506641,5089.139469,5588.291564
433752,546364,0,242.131013,True,1202.0,Non-CBSA Middle Atlantic,5A,5A,Hour4,2,...,0.000000,0.0,8554.681791,0.0,0.000000,0.0,3479.581913,7804.239977,6993.834612,8554.681791


## Baseline Energy Consumption

In [6]:
print("""
-------------------------------------------------------------------------------------------------------
Baseline Consumption:
-------------------------------------------------------------------------------------------------------
""")

# df_baseline_enduse(df_baseline, df_enduse, category, fuel_filter='Yes', tech_filter='Yes')
df_euss_am_baseline_home = df_enduse_refactored(df_baseline = df_euss_am_baseline,
                                                fuel_filter = 'Yes',
                                                tech_filter = 'Yes')
df_euss_am_baseline_home


-------------------------------------------------------------------------------------------------------
Baseline Consumption:
-------------------------------------------------------------------------------------------------------

Processing column: in.clothes_dryer
Initial data types: object
Data types after processing: object
Processing column: in.cooking_range
Initial data types: object
Data types after processing: object
329 rows remain after applying total heating consumption calculation
Filtered for the following fuels: ['Natural Gas', 'Electricity', 'Propane', 'Fuel Oil']
328 rows remain after applying heating fuel filter
Filtered for the following Heating technologies: ['Electricity ASHP', 'Electricity Baseboard', 'Electricity Electric Boiler', 'Electricity Electric Furnace', 'Fuel Oil Fuel Boiler', 'Fuel Oil Fuel Furnace', 'Natural Gas Fuel Boiler', 'Natural Gas Fuel Furnace', 'Propane Fuel Boiler', 'Propane Fuel Furnace']
275 rows remain after applying heating technology fil

,bldg_id,square_footage,census_region,building_america_climate_zone,cambium_GEA_region,state,city,county,puma,county_and_puma,...,base_naturalGas_clothesDrying_consumption,base_propane_clothesDrying_consumption,base_cooking_fuel,base_electricity_cooking_consumption,base_naturalGas_cooking_consumption,base_propane_cooking_consumption,baseline_heating_consumption,baseline_waterHeating_consumption,baseline_clothesDrying_consumption,baseline_cooking_consumption
410884,1081,1690.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001702,"G4200030, G42001702",...,0.000000,0.0,Natural Gas,27.841752,813.565290,0.0,53282.078952,3896.966018,400.042011,841.407042
411011,4075,2152.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001701,"G4200030, G42001701",...,439.313534,0.0,Electricity,389.491452,0.000000,0.0,28846.399278,3711.452030,472.430565,389.491452
411058,5187,1220.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001702,"G4200030, G42001702",...,0.000000,0.0,Natural Gas,27.841752,813.565290,0.0,25124.396689,4021.228151,400.042011,841.407042
411079,5617,1690.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001701,"G4200030, G42001701",...,0.000000,0.0,Natural Gas,27.841752,813.565290,0.0,32458.500216,4201.173788,400.042011,841.407042
411192,7867,1220.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001702,"G4200030, G42001702",...,0.000000,0.0,Electricity,324.429674,0.000000,0.0,28341.144753,1402.345070,320.033608,324.429674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433429,538404,8194.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001701,"G4200030, G42001701",...,918.191662,0.0,Electricity,603.140262,0.000000,0.0,115028.636549,7341.137232,987.356435,603.140262
433435,538501,1220.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001702,"G4200030, G42001702",...,327.360385,0.0,Natural Gas,23.152615,679.631811,0.0,22626.845030,2788.571231,351.978355,702.784426
433547,541686,1690.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001702,"G4200030, G42001702",...,0.000000,0.0,Natural Gas,33.410102,976.219734,0.0,22535.406857,6511.159962,480.050413,1009.629836
433583,542571,1220.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001701,"G4200030, G42001701",...,327.360385,0.0,Natural Gas,23.152615,679.631811,0.0,10601.552886,1137.115752,351.978355,702.784426


## Public Perspective: Monetized Marginal Damages from Emissions

### Step 1: Calculate emissions factors for different fuel sources

### Marginal Emissions Factors
#### Electricity
- STATE Regional Aggregation is what is used in the Parth Analysis 
- "Marginal Emissions Factors for Electricity"
- Factor Type: Marginal
- Calculation Method: Regression
- Metric: Emissions [kg/MWh]")
- Predictor: Year")
- Pollutants: SO2, NOx, PM2.5, CO2")
#### Fossil Fuels
- NOx, SO2, CO2: 
    - RESNET Table 7.1.2 Emissions Factors for Household Combustion Fuels
    - Source: https://www.resnet.us/wp-content/uploads/ANSIRESNETICC301-2022_resnetpblshd.pdf
    - All factors are in units of lb/Mbtu so energy consumption in kWh need to be converted to kWh 
    - (1 lb / Mbtu) * (1 Mbtu / 1x10^6 Btu) * (3412 Btu / 1 kWh)
- PM2.5: 
    - A National Methodology and Emission Inventory for Residential Fuel Combustion
    - Source: https://www3.epa.gov/ttnchie1/conference/ei12/area/haneke.pdf

In [7]:
print("""
-------------------------------------------------------------------------------------------------------
Public Perspective: Monetized Marginal Damages from Emissions
-------------------------------------------------------------------------------------------------------
Step 1: Calculate emissions factors for different fuel sources
- Electricity
- Natural Gas
- Fuel Oil 
- Propane
-------------------------------------------------------------------------------------------------------
""")


-------------------------------------------------------------------------------------------------------
Public Perspective: Monetized Marginal Damages from Emissions
-------------------------------------------------------------------------------------------------------
Step 1: Calculate emissions factors for different fuel sources
- Electricity
- Natural Gas
- Fuel Oil 
- Propane
-------------------------------------------------------------------------------------------------------



In [8]:
print("""
-------------------------------------------------------------------------------------------------------
Calculate Emissions Factors: ELECTRICITY
-------------------------------------------------------------------------------------------------------
Electricity Marginal Emissions Factors:
- STATE Regional Aggregation is what is used in the Parth Analysis 
- "Marginal Emissions Factors for Electricity"
- Factor Type: Marginal
- Calculation Method: Regression
- Metric: Emissions [kg/MWh]
- Predictor: Year")
- Pollutants: SO2, NOx, PM2.5, CO2
-------------------------------------------------------------------------------------------------------
""")
filename = 'Generation-MARREG-EMIT-state-byYear.csv'
relative_path = os.path.join(r"margEmis_electricity", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_margEmissions = pd.read_csv(file_path, index_col=0)

# Convert from kg/MWh to lb/kWh
# Obtain value from the CSV file and convert to lbs pollutant per kWh 
df_margEmis_electricity = pd.DataFrame({
    'state': df_margEmissions['region'],
    'fuel_type': 'electricity',
    'pollutant': df_margEmissions['pollutant'],
    'value': df_margEmissions['factor'] * (2.20462/1) * (1/1000),
    'unit': '[lb/kWh]'
})
df_margEmis_electricity


-------------------------------------------------------------------------------------------------------
Calculate Emissions Factors: ELECTRICITY
-------------------------------------------------------------------------------------------------------
Electricity Marginal Emissions Factors:
- STATE Regional Aggregation is what is used in the Parth Analysis 
- "Marginal Emissions Factors for Electricity"
- Factor Type: Marginal
- Calculation Method: Regression
- Metric: Emissions [kg/MWh]
- Predictor: Year")
- Pollutants: SO2, NOx, PM2.5, CO2
-------------------------------------------------------------------------------------------------------

Retrieved data for filename: Generation-MARREG-EMIT-state-byYear.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\margEmis_electricity\Generation-MARREG-EMIT-state-byYear.csv




,state,fuel_type,pollutant,value,unit
1,AL,electricity,so2,0.000131,[lb/kWh]
2,AL,electricity,nox,0.000440,[lb/kWh]
3,AL,electricity,pm25,0.000140,[lb/kWh]
4,AL,electricity,co2,1.172667,[lb/kWh]
5,AR,electricity,so2,0.002555,[lb/kWh]
...,...,...,...,...,...
184,WV,electricity,co2,1.713400,[lb/kWh]
185,WY,electricity,so2,0.001223,[lb/kWh]
186,WY,electricity,nox,0.001602,[lb/kWh]
187,WY,electricity,pm25,0.000259,[lb/kWh]


In [9]:
print("""
-------------------------------------------------------------------------------------------------------
Calculate Emissions Factors: FOSSIL FUELS
-------------------------------------------------------------------------------------------------------
Fossil Fuels (Natural Gas, Fuel Oil, Propane):
- NOx, SO2, CO2: 
    - RESNET Table 7.1.2 Emissions Factors for Household Combustion Fuels
    - Source: https://www.resnet.us/wp-content/uploads/ANSIRESNETICC301-2022_resnetpblshd.pdf
    - All factors are in units of lb/Mbtu so energy consumption in kWh need to be converted to kWh 
    - (1 lb / Mbtu) * (1 Mbtu / 1x10^6 Btu) * (3412 Btu / 1 kWh)
- PM2.5: 
    - A National Methodology and Emission Inventory for Residential Fuel Combustion
    - Source: https://www3.epa.gov/ttnchie1/conference/ei12/area/haneke.pdf
-------------------------------------------------------------------------------------------------------
""")

fuelOil_factors = calculate_fossilFuel_emission_factor("fuelOil", 0.0015, 0.1300, 0.83, 161.0, 1000, 138500)
naturalGas_factors = calculate_fossilFuel_emission_factor("naturalGas", 0.0006, 0.0922, 1.9, 117.6, 1000000, 1039)
propane_factors = calculate_fossilFuel_emission_factor("propane", 0.0002, 0.1421, 0.17, 136.6, 1000, 91452)

all_factors = {**fuelOil_factors, **naturalGas_factors, **propane_factors}

df_margEmis_factors = pd.DataFrame.from_dict(all_factors, orient="index", columns=["value"])
df_margEmis_factors.reset_index(inplace=True)
df_margEmis_factors.columns = ["pollutant", "value"]
df_margEmis_factors[["fuel_type", "pollutant"]] = df_margEmis_factors["pollutant"].str.split("_", expand=True)
df_margEmis_factors["unit"] = "[lb/kWh]"

# Add the 'state' column and assign 'National' to every row
df_margEmis_factors = df_margEmis_factors.assign(state='National')

df_margEmis_factors = df_margEmis_factors[["state", "fuel_type", "pollutant", "value", "unit"]]
df_margEmis_factors


-------------------------------------------------------------------------------------------------------
Calculate Emissions Factors: FOSSIL FUELS
-------------------------------------------------------------------------------------------------------
Fossil Fuels (Natural Gas, Fuel Oil, Propane):
- NOx, SO2, CO2: 
    - RESNET Table 7.1.2 Emissions Factors for Household Combustion Fuels
    - Source: https://www.resnet.us/wp-content/uploads/ANSIRESNETICC301-2022_resnetpblshd.pdf
    - All factors are in units of lb/Mbtu so energy consumption in kWh need to be converted to kWh 
    - (1 lb / Mbtu) * (1 Mbtu / 1x10^6 Btu) * (3412 Btu / 1 kWh)
- PM2.5: 
    - A National Methodology and Emission Inventory for Residential Fuel Combustion
    - Source: https://www3.epa.gov/ttnchie1/conference/ei12/area/haneke.pdf
-------------------------------------------------------------------------------------------------------



,state,fuel_type,pollutant,value,unit
0,National,fuelOil,so2,5.118000e-06,[lb/kWh]
1,National,fuelOil,nox,4.435600e-04,[lb/kWh]
2,National,fuelOil,pm25,2.044736e-05,[lb/kWh]
3,National,fuelOil,co2,5.493320e-01,[lb/kWh]
4,National,naturalGas,so2,2.047200e-06,[lb/kWh]
5,National,naturalGas,nox,3.145864e-04,[lb/kWh]
6,National,naturalGas,pm25,6.239461e-06,[lb/kWh]
7,National,naturalGas,co2,4.012512e-01,[lb/kWh]
8,National,propane,so2,6.824000e-07,[lb/kWh]
9,National,propane,nox,4.848452e-04,[lb/kWh]


### Step 2: Adjust Natural Gas & Electricity Emissions Factors for Natural Gas Leakage

In [10]:
print("""
-------------------------------------------------------------------------------------------------------
Step 2: Adjust Natural Gas & Electricity Emissions Factors for Natural Gas Leakage
-------------------------------------------------------------------------------------------------------
Natural Gas (Deetjen et al.): 
"To account for the natural gas infrastructure's leakage of the greenhouse gas methane, 
we estimate the amount of methane leaked per therm of natural gas consumed for heating and 
convert to CO2-equivalent emissions via the GWP of methane. We assume that for every therm of 
natural gas consumed for heating, 0.023 therms of methane escape to the atmosphere [28]. 
Using the energy density of natural gas, we convert from therms to kilograms and multiply 
by 28—the GWP of methane [29]—to calculate a rate of 1.27 kg CO2-equivalent per therm of 
consumed natural gas."

Electricity NERC Regions (Deetjen et al): 
"To account for the natural gas infrastructure's leakage of the greenhouse gas methane, 
we estimate the amount of methane leaked per MWh of electricity generation in each NERC 
region and convert to CO2-equivalent emissions via the global warming potential (GWP) of methane. 
For example, we find that in 2017, the states comprising the western region (WECC) of 
the US electric grid consumed 1.45 million MMcf of natural gas in the power sector [27]. 
We assume that for every MMcf of consumed natural gas, 0.023 MMcf of methane is leaked into 
the atmosphere [28]. By multiplying that leakage rate by the 1.45 million MMcf of consumed 
natural gas, converting to tonnes, and multiplying by a GWP of 28 [29], we estimate 
that the 2017 WECC power sector contributed to methane leakage amounting to 18.6 Mt CO2-equivalent.
By dividing this 18.6 Mt by the 724 TWh of the WECC states' generated electricity [27], we 
calculate a methane leakage rate factor of 25.7 kg MWh−1. In the same manner, we calculate the 
methane leakage rate factors for the other NERC regions. We use the 100 years GWP value of 28 
for methane. Although there have been proposals to use 20 years GWP values, recent research 
shows that the benefits of this alternative 20 years time from are overstated [30]."
-------------------------------------------------------------------------------------------------------
""")
filename = 'natural_gas_leakage_rate.csv'
relative_path = os.path.join(r"margEmis_electricity", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_naturalGas_leakage_rate = pd.read_csv(file_path)

state_abbreviations = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'District of Columbia': 'DC',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# Map full state names to abbreviations
df_naturalGas_leakage_rate['state'] = df_naturalGas_leakage_rate['state_name'].map(state_abbreviations)

# thousand Mcf * (0.023 Mcf leak/1 Mcf) * (19.3 tonnes/1000 Mcf) * (1000 kg/1 tonne) * (2.205 lb/1 kg)) / (thousand MWh * (1000 MWh/thousand MWh)) 
df_naturalGas_leakage_rate['naturalGas_leakage_lbCH4_perMWh'] = (df_naturalGas_leakage_rate['naturalGas_electricity_generation'] * (0.023/1) * (19.3/1) * (1000/1) * (2.205/1)) / (df_naturalGas_leakage_rate['net_generation'] * (1000/1)) 

# (lb CH4/MWh) * (28 lb CO2e/1 lb CH4)
df_naturalGas_leakage_rate['naturalGas_leakage_lbCO2e_perMWh'] = df_naturalGas_leakage_rate['naturalGas_leakage_lbCH4_perMWh'] * (28/1)

# (lb CO2e/MWh) * (1 MWh / 1000 kWh)
df_naturalGas_leakage_rate['naturalGas_leakage_lbCO2e_perkWh'] = df_naturalGas_leakage_rate['naturalGas_leakage_lbCO2e_perMWh'] * (1/1000)
df_naturalGas_leakage_rate


-------------------------------------------------------------------------------------------------------
Step 2: Adjust Natural Gas & Electricity Emissions Factors for Natural Gas Leakage
-------------------------------------------------------------------------------------------------------
Natural Gas (Deetjen et al.): 
"To account for the natural gas infrastructure's leakage of the greenhouse gas methane, 
we estimate the amount of methane leaked per therm of natural gas consumed for heating and 
convert to CO2-equivalent emissions via the GWP of methane. We assume that for every therm of 
natural gas consumed for heating, 0.023 therms of methane escape to the atmosphere [28]. 
Using the energy density of natural gas, we convert from therms to kilograms and multiply 
by 28—the GWP of methane [29]—to calculate a rate of 1.27 kg CO2-equivalent per therm of 
consumed natural gas."

Electricity NERC Regions (Deetjen et al): 
"To account for the natural gas infrastructure's leakage of the

,state_name,naturalGas_electricity_generation,units,net_generation,units.1,state,naturalGas_leakage_lbCH4_perMWh,naturalGas_leakage_lbCO2e_perMWh,naturalGas_leakage_lbCO2e_perkWh
0,Connecticut,135274,thousand Mcf,38376,thousand megawatthours,CT,3.450233,96.606511,0.096607
1,Maine,13718,thousand Mcf,9308,thousand megawatthours,ME,1.442541,40.391148,0.040391
2,Massachusetts,128810,thousand Mcf,26263,thousand megawatthours,MA,4.800638,134.417872,0.134418
3,New Hampshire,21563,thousand Mcf,16988,thousand megawatthours,NH,1.242398,34.787138,0.034787
4,Rhode Island,57260,thousand Mcf,8170,thousand megawatthours,RI,6.859983,192.079518,0.192080
5,Vermont,11,thousand Mcf,2175,thousand megawatthours,VT,0.004950,0.138607,0.000139
6,New Jersey,267122,thousand Mcf,73727,thousand megawatthours,NJ,3.546311,99.296711,0.099297
7,New York,390742,thousand Mcf,130301,thousand megawatthours,NY,2.935189,82.185295,0.082185
8,Pennsylvania,530196,thousand Mcf,212285,thousand megawatthours,PA,2.444617,68.449284,0.068449
9,Illinois,135512,thousand Mcf,184791,thousand megawatthours,IL,0.717779,20.097809,0.020098


In [11]:
# NATURAL GAS LEAKAGE: NATURAL GAS USED IN ELECTRICITY GENERATION
if 'naturalGas_leakage_lbCO2e_perkWh' in df_margEmis_electricity.columns:
    df_margEmis_electricity.drop(columns=['naturalGas_leakage_lbCO2e_perkWh'], inplace=True)

df_margEmis_electricity = df_margEmis_electricity.merge(
    df_naturalGas_leakage_rate[['state', 'naturalGas_leakage_lbCO2e_perkWh']],
    how='left',  # Use a left join to keep all rows from df_margEmis_electricity
    on=['state']  # Merge on the 'state' column
)
# Set 'naturalGas_leakage_lbCO2e_perkWh' to zero where 'pollutant' is not 'co2'
df_margEmis_electricity.loc[df_margEmis_electricity['pollutant'] != 'co2', 'naturalGas_leakage_lbCO2e_perkWh'] = 0.0

# Calculate adjusted marginal emissions factore with natural gas fugitive emissions
df_margEmis_electricity['margEmis_factor_adjusted'] = df_margEmis_electricity['value'] + df_margEmis_electricity['naturalGas_leakage_lbCO2e_perkWh'] 

# Create a factor to multiply marginal damages by
df_margEmis_electricity['naturalGas_leakage_factor'] = df_margEmis_electricity['margEmis_factor_adjusted'] / df_margEmis_electricity['value']

# Reorder columns
df_margEmis_electricity = df_margEmis_electricity[['state', 'fuel_type', 'pollutant', 'value', 'unit', 'naturalGas_leakage_lbCO2e_perkWh', 'margEmis_factor_adjusted', 'naturalGas_leakage_factor']]
df_margEmis_electricity

,state,fuel_type,pollutant,value,unit,naturalGas_leakage_lbCO2e_perkWh,margEmis_factor_adjusted,naturalGas_leakage_factor
0,AL,electricity,so2,0.000131,[lb/kWh],0.000000,0.000131,1.000000
1,AL,electricity,nox,0.000440,[lb/kWh],0.000000,0.000440,1.000000
2,AL,electricity,pm25,0.000140,[lb/kWh],0.000000,0.000140,1.000000
3,AL,electricity,co2,1.172667,[lb/kWh],0.080872,1.253539,1.068964
4,AR,electricity,so2,0.002555,[lb/kWh],0.000000,0.002555,1.000000
...,...,...,...,...,...,...,...,...
183,WV,electricity,co2,1.713400,[lb/kWh],0.004440,1.717840,1.002591
184,WY,electricity,so2,0.001223,[lb/kWh],0.000000,0.001223,1.000000
185,WY,electricity,nox,0.001602,[lb/kWh],0.000000,0.001602,1.000000
186,WY,electricity,pm25,0.000259,[lb/kWh],0.000000,0.000259,1.000000


In [12]:
# NATURAL GAS LEAKAGE: NATURAL GAS INFRASTRUCTURE
# leakage rate for natural gas infrastructure
# 1 Therm = 29.30 kWh --> 1.27 kg CO2e/therm * (1 therm/29.30 kWh) = 0.043 kg CO2e/kWh = 0.095 lb CO2e/kWh
df_margEmis_factors['naturalGas_leakage_lbCO2e_perkWh'] = 0.095

# Set 'naturalGas_leakage_lbCO2e_perkWh' to zero where 'pollutant' is not 'co2'
df_margEmis_factors.loc[df_margEmis_factors['pollutant'] != 'co2', 'naturalGas_leakage_lbCO2e_perkWh'] = 0.0

# Set 'naturalGas_leakage_lbCO2e_perkWh' to zero where 'fuel_type' is not 'naturalGas'
df_margEmis_factors.loc[df_margEmis_factors['fuel_type'] != 'naturalGas', 'naturalGas_leakage_lbCO2e_perkWh'] = 0.0

# Calculate adjusted marginal emissions factor with natural gas fugitive emissions
df_margEmis_factors['margEmis_factor_adjusted'] = df_margEmis_factors['value'] + df_margEmis_factors['naturalGas_leakage_lbCO2e_perkWh'] 

# Create a factor to multiply marginal damages by
df_margEmis_factors['naturalGas_leakage_factor'] = df_margEmis_factors['margEmis_factor_adjusted'] / df_margEmis_factors['value']

# Reorder columns
df_margEmis_factors = df_margEmis_factors[['state', 'fuel_type', 'pollutant', 'value', 'unit', 'naturalGas_leakage_lbCO2e_perkWh', 'margEmis_factor_adjusted', 'naturalGas_leakage_factor']]
df_margEmis_factors

,state,fuel_type,pollutant,value,unit,naturalGas_leakage_lbCO2e_perkWh,margEmis_factor_adjusted,naturalGas_leakage_factor
0,National,fuelOil,so2,5.118000e-06,[lb/kWh],0.000,5.118000e-06,1.000000
1,National,fuelOil,nox,4.435600e-04,[lb/kWh],0.000,4.435600e-04,1.000000
2,National,fuelOil,pm25,2.044736e-05,[lb/kWh],0.000,2.044736e-05,1.000000
3,National,fuelOil,co2,5.493320e-01,[lb/kWh],0.000,5.493320e-01,1.000000
4,National,naturalGas,so2,2.047200e-06,[lb/kWh],0.000,2.047200e-06,1.000000
5,National,naturalGas,nox,3.145864e-04,[lb/kWh],0.000,3.145864e-04,1.000000
6,National,naturalGas,pm25,6.239461e-06,[lb/kWh],0.000,6.239461e-06,1.000000
7,National,naturalGas,co2,4.012512e-01,[lb/kWh],0.095,4.962512e-01,1.236759
8,National,propane,so2,6.824000e-07,[lb/kWh],0.000,6.824000e-07,1.000000
9,National,propane,nox,4.848452e-04,[lb/kWh],0.000,4.848452e-04,1.000000


In [13]:
# Append df_margEmissions_electricity to df_margEmis_factors
# This produces a dataframe of marginal emissions rates for various fuel types
df_margEmis_factors = pd.concat([df_margEmis_factors, df_margEmis_electricity], ignore_index=True)
df_margEmis_factors

,state,fuel_type,pollutant,value,unit,naturalGas_leakage_lbCO2e_perkWh,margEmis_factor_adjusted,naturalGas_leakage_factor
0,National,fuelOil,so2,0.000005,[lb/kWh],0.000000,0.000005,1.000000
1,National,fuelOil,nox,0.000444,[lb/kWh],0.000000,0.000444,1.000000
2,National,fuelOil,pm25,0.000020,[lb/kWh],0.000000,0.000020,1.000000
3,National,fuelOil,co2,0.549332,[lb/kWh],0.000000,0.549332,1.000000
4,National,naturalGas,so2,0.000002,[lb/kWh],0.000000,0.000002,1.000000
...,...,...,...,...,...,...,...,...
195,WV,electricity,co2,1.713400,[lb/kWh],0.004440,1.717840,1.002591
196,WY,electricity,so2,0.001223,[lb/kWh],0.000000,0.001223,1.000000
197,WY,electricity,nox,0.001602,[lb/kWh],0.000000,0.001602,1.000000
198,WY,electricity,pm25,0.000259,[lb/kWh],0.000000,0.000259,1.000000


### Step 3: Quantify monitized damages using EASIUR Marginal Social Cost Factors
#### THE STEPS BELOW SUMMARIZE WHAT WAS DONE TO OBTAIN ALL NATIONAL EASIUR VALUES INCLUDED ON GITHUB
- Obtain all of the dwelling unit latitude and longitude values from the metadata columns
- Make a new dataframe of just the longitude and latitude values 
    - Make sure that the order is (longitude, latitude)
    - Do not include the index or column name when exporting 
- Export the CSV
- **Upload csv to EASIUR Website:**
    - Website: https://barney.ce.cmu.edu/~jinhyok/easiur/online/
    - See inputs in respective sections
- Download the file and put it in the 'easiur_batchConversion_download' folder
- Copy and paste the name of the file EASIUR generated when prompted
- Copy and paste the name of the filepath for the 'easiur_batchConversion_download' folder when prompted
- Match up the longitude and latitudes for each dwelling unit with the selected damages

### Fossil Fuels: EASIUR Marginal Damage (Social Cost) Factors Info
- Factor Type: Marginal Social Cost
- Calculation Method: Regression
- Metric: Marginal Social Cost [USD per metric ton]
- Dollar Year: 2010
- Income Year: 2018
- Population Year: 2018
- Aggregation: Longitude, and Latitude Coordinates
- Pollutants: Primary PM2.5, Sulfur Dioxide (SO2), Nitrogen Oxides (NOx), Ammonia (NH3)
- Elevation (Ground, 150m, 300m) and Seasons (Winter, Spring, Summer, Fall)

In [14]:
# Create a dataframe containing just the longitude and Latitude
df_EASIUR_batchConversion = pd.DataFrame({
    'Longitude':df_euss_am_baseline['in.weather_file_longitude'],
    'Latitude':df_euss_am_baseline['in.weather_file_latitude'],
})

# Drop duplicate rows based on 'Longitude' and 'Latitude' columns
df_EASIUR_batchConversion.drop_duplicates(subset=['Longitude', 'Latitude'], keep='first', inplace=True)

# Create a location ID for the name of the batch conversion file
while True:
    if menu_state == 'N':
        location_id = 'National'
        print("You chose to analyze all of the United States.")
        break
    elif menu_state == 'Y':
        if menu_city == 'N':
            try:
                location_id = str(input_state)
                print(f"Location ID is: {location_id}")
                break
            except ValueError:
                print("Invalid input for state!")
        elif menu_city == 'Y':
            try:
                location_id = input_cityFilter.replace(', ', '_').strip()
                print(f"Location ID is: {location_id}")
                break
            except AttributeError:
                print("Invalid input for city filter!")
        else:
            print("Incorrect state or city filter assignment!")
    else:
        print("Invalid data location. Check your inputs at the beginning of this notebook!")

# Updated GitHub code has EASIUR file with all unique latitude, longitude coordinates in the US
filename = 'easiur_National2024-06-1420-17.csv'
relative_path = os.path.join(r"margDamages_EASIUR\easiur_batchConversion_download", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_margSocialCosts = pd.read_csv(file_path)

# Convert from kg/MWh to lb/kWh
# Obtain value from the CSV file and convert to lbs pollutant per kWh 
# Inflate from 2010 to 2018

# Define df_margSocialCosts_EASIUR DataFrame first
df_margSocialCosts_EASIUR = pd.DataFrame({
    'Longitude': df_margSocialCosts['Longitude'],
    'Latitude': df_margSocialCosts['Latitude']
})
df_margSocialCosts_EASIUR

Location ID is: Pittsburgh
Retrieved data for filename: easiur_National2024-06-1420-17.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\margDamages_EASIUR\easiur_batchConversion_download\easiur_National2024-06-1420-17.csv




,Longitude,Latitude
0,-87.04,31.42
1,-85.86,33.59
2,-86.39,32.30
3,-85.45,31.32
4,-86.75,33.56
...,...,...
1024,-106.72,44.38
1025,-105.54,44.34
1026,-107.95,43.97
1027,-108.08,44.52


### Step 4: Inflate Marginal Social Cost (Damage) Factors using BLS CPI for All Urban Consumers (CPI-U)
- Series Id:	CUUR0000SA0
- Not Seasonally Adjusted
- Series Title:	All items in U.S. city average, all urban consumers, not seasonally adjusted
- Area:	U.S. city average
- Item:	All items
- Base Period:	1982-84=100

### Use the updated Social Cost of Carbon (190 USD-2020/ton CO2) and inflate to USD-2021
- EPA Median for 2% near term discount rate and most commonly mentioned value is 190 USD-2020 using the GIVE model.
- 190 USD-2020 has some inconsistency with the VSL being used. An old study and 2008 VSL is noted
- 190 USD value and inflate to USD 2021 because there is a clear source and ease of replicability.

### Adjustment for VSL
- EASIUR uses a VSL of 8.8M USD-2010 
- New EPA VSL is 11.3M USD-2021

In [15]:
# Load the BLS Inflation Data
filename = 'bls_cpiu_2005-2023.xlsx'
relative_path = os.path.join(r"inflation_data", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# Create a pandas dataframe
df_bls_cpiu = pd.read_excel(file_path, sheet_name='bls_cpiu')

df_bls_cpiu = pd.DataFrame({
    'year': df_bls_cpiu['Year'],
    'cpiu_annual': df_bls_cpiu['Annual']
})

# Obtain the Annual CPIU values for the years of interest
bls_cpi_annual_2008 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2008)].item()
bls_cpi_annual_2010 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2010)].item()
bls_cpi_annual_2013 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2013)].item()
bls_cpi_annual_2018 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2018)].item()
bls_cpi_annual_2019 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2019)].item()
bls_cpi_annual_2020 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2020)].item()
bls_cpi_annual_2021 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2021)].item()
bls_cpi_annual_2022 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2022)].item()

# Precompute constant values
cpi_ratio_2021_2021 = bls_cpi_annual_2021 / bls_cpi_annual_2021  # This will be 1
cpi_ratio_2021_2020 = bls_cpi_annual_2021 / bls_cpi_annual_2020  # For SCC
cpi_ratio_2021_2019 = bls_cpi_annual_2021 / bls_cpi_annual_2019 
cpi_ratio_2021_2018 = bls_cpi_annual_2021 / bls_cpi_annual_2018 
cpi_ratio_2021_2013 = bls_cpi_annual_2021 / bls_cpi_annual_2013
cpi_ratio_2021_2010 = bls_cpi_annual_2021 / bls_cpi_annual_2010
cpi_ratio_2021_2008 = bls_cpi_annual_2021 / bls_cpi_annual_2008  # For EPA VSL and SCC

# 2021 US EPA VSL is $11.3M in 2021 USD
df_margSocialCosts_EASIUR['current_VSL_USD2021'] = 11.3

# Easiur uses a VSL of $8.8 M USD2010
# Inflate to 2021 $USD
df_margSocialCosts_EASIUR['easiur_VSL_USD2021'] = 8.8 * cpi_ratio_2021_2010

# Use df_margSocialCosts_EASIUR in the calculation of other columns
# Also adjust the VSL
df_margSocialCosts_EASIUR['margSocialCosts_pm25'] = round((df_margSocialCosts['PM25 Annual Ground'] * (1/2204.6) * (df_margSocialCosts_EASIUR['current_VSL_USD2021']/df_margSocialCosts_EASIUR['easiur_VSL_USD2021'])), 2)
df_margSocialCosts_EASIUR['margSocialCosts_so2'] = round((df_margSocialCosts['SO2 Annual Ground'] * (1/2204.6) * (df_margSocialCosts_EASIUR['current_VSL_USD2021']/df_margSocialCosts_EASIUR['easiur_VSL_USD2021'])), 2)
df_margSocialCosts_EASIUR['margSocialCosts_nox'] = round((df_margSocialCosts['NOX Annual Ground'] * (1/2204.6) * (df_margSocialCosts_EASIUR['current_VSL_USD2021']/df_margSocialCosts_EASIUR['easiur_VSL_USD2021'])), 2)

# Note that SCC of $190 USD-2020 has some inconsistency with the VSL being used. An old study and 2008 VSL is noted
# We use the $190 USD value and inflate to USD 2021 because there is a clear source and ease of replicability.
df_margSocialCosts_EASIUR['margSocialCosts_co2'] = round((190 * cpi_ratio_2021_2020 * (1/2204.6)), 2)
df_margSocialCosts_EASIUR['unit'] = '[$USD2021/lb]'
df_margSocialCosts_EASIUR

Retrieved data for filename: bls_cpiu_2005-2023.xlsx
Located at filepath: c:\Users\14128\Research\cmu-tare-model\inflation_data\bls_cpiu_2005-2023.xlsx


,Longitude,Latitude,current_VSL_USD2021,easiur_VSL_USD2021,margSocialCosts_pm25,margSocialCosts_so2,margSocialCosts_nox,margSocialCosts_co2,unit
0,-87.04,31.42,11.3,10.935429,36.37,8.72,1.63,0.09,[$USD2021/lb]
1,-85.86,33.59,11.3,10.935429,54.55,9.56,2.53,0.09,[$USD2021/lb]
2,-86.39,32.30,11.3,10.935429,49.71,9.08,2.05,0.09,[$USD2021/lb]
3,-85.45,31.32,11.3,10.935429,41.54,9.09,1.79,0.09,[$USD2021/lb]
4,-86.75,33.56,11.3,10.935429,94.13,10.51,2.89,0.09,[$USD2021/lb]
...,...,...,...,...,...,...,...,...,...
1024,-106.72,44.38,11.3,10.935429,9.87,4.69,0.74,0.09,[$USD2021/lb]
1025,-105.54,44.34,11.3,10.935429,10.93,5.64,0.93,0.09,[$USD2021/lb]
1026,-107.95,43.97,11.3,10.935429,8.51,4.20,0.59,0.09,[$USD2021/lb]
1027,-108.08,44.52,11.3,10.935429,8.90,5.37,0.79,0.09,[$USD2021/lb]


## Electricity CEDM-EASIUR Marginal Damages: Current and Decarbonizing Grid
- Factor Type: Marginal
- Calculation Method: Regression
- Metric: Marginal Damages EASIUR [USD per MWh or kWh]
- Year: 2018
- Regional Aggregation: eGRID subregion (all regions)
- Pollutants: SO2, NOx, PM2.5 CO2

SCC Adjustment: We use the EPA suggested 190 USD-2020 value for the social cost of carbon and inflate to 2021 USD. 

VSL: "We use a value of a statistical life (VSL) of USD 8.8 million (in 2010 dollars) for both our AP2 and EASIUR calculations. EASIUR reports damage intensities in USD/metric ton using this VSL and dollar year."

In [16]:
# For CO2 adjust SCC
# Create an adjustment factor for the new Social Cost of Carbon (SCC)
epa_scc = 190 * cpi_ratio_2021_2020
old_scc = 40 * cpi_ratio_2021_2010
scc_adjustment_factor = epa_scc / old_scc

# For Health-Related Emissions Adjust for different Value of a Statistical Life (VSL) values
# Current VSL is $11.3 M USD2021
current_VSL_USD2021 = 11.3

# Easiur uses a VSL of $8.8 M USD2010
easiur_VSL_USD2021 = 8.8 * (cpi_ratio_2021_2010)

# Calculate VSL adjustment factor
vsl_adjustment_factor = current_VSL_USD2021 / easiur_VSL_USD2021

### Damages from Climate Related Emissions

In [17]:
# Climate damages (co2) are expected to decline 68% linearlly by 2030 (% relative to 2005)
# Note only 2006 data available, used in place of 2005
filename = 'Generation-MARREG-DAMEASIUR-egrid-byYear_climate2006.csv'
relative_path = os.path.join(r"margDamages_EASIUR", filename)
file_path = os.path.join(project_root, relative_path)
df_margDamages_climate2006 = pd.read_csv(file_path, index_col=0)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

Retrieved data for filename: Generation-MARREG-DAMEASIUR-egrid-byYear_climate2006.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\margDamages_EASIUR\Generation-MARREG-DAMEASIUR-egrid-byYear_climate2006.csv


In [18]:
# Climate damages (co2) are expected to decline 68% linearlly by 2030 (% relative to 2005)
# Note 2018 start year
filename = 'Generation-MARREG-DAMEASIUR-egrid-byYear_climate2018.csv'
relative_path = os.path.join(r"margDamages_EASIUR", filename)
file_path = os.path.join(project_root, relative_path)
df_margDamages_climate2018 = pd.read_csv(file_path, index_col=0)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

Retrieved data for filename: Generation-MARREG-DAMEASIUR-egrid-byYear_climate2018.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\margDamages_EASIUR\Generation-MARREG-DAMEASIUR-egrid-byYear_climate2018.csv


In [19]:
# Marginal damages [$/kWh]
# Inflate from 2010 to 2021
# Note only 2006 data available, used in place of 2005
df_margDamages_EASIUR_climate = pd.DataFrame({
    'subregion_eGRID': df_margDamages_climate2006['region'],
    'pollutant': df_margDamages_climate2006['pollutant'],
    'unit': '[$/kWh]',
    '2030_decarb': '68% from 2005',
    'margDamages_dollarPerkWh_adjustVSL_ref': (df_margDamages_climate2006['factor'] * (scc_adjustment_factor) * (1/1000)) * (cpi_ratio_2021_2010),
    'margDamages_dollarPerkWh_adjustVSL_2018': (df_margDamages_climate2018['factor'] * (scc_adjustment_factor) * (1/1000)) * (cpi_ratio_2021_2010)
})
df_margDamages_EASIUR_climate['margDamages_decarb_2030'] = df_margDamages_EASIUR_climate['margDamages_dollarPerkWh_adjustVSL_ref'] - (df_margDamages_EASIUR_climate['margDamages_dollarPerkWh_adjustVSL_ref'] * 0.68)
df_margDamages_EASIUR_climate['reduction_margDamages_2030'] = df_margDamages_EASIUR_climate['margDamages_dollarPerkWh_adjustVSL_2018'] - df_margDamages_EASIUR_climate['margDamages_decarb_2030']
df_margDamages_EASIUR_climate['reduction_margDamages_annual'] = df_margDamages_EASIUR_climate['reduction_margDamages_2030'] / 12 # Relative to 2018, 
# df_margDamages_EASIUR_climate

### Damages from Health Related Emissions

In [20]:
# Health damages (SO2, NOx, PM2.5) are expected to decline 65% by 2030 (% relative from 2021)
filename = 'Generation-MARREG-DAMEASIUR-egrid-byYear_health2018.csv'
relative_path = os.path.join(r"margDamages_EASIUR", filename)
file_path = os.path.join(project_root, relative_path)
df_margDamages_health2018 = pd.read_csv(file_path, index_col=0)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

Retrieved data for filename: Generation-MARREG-DAMEASIUR-egrid-byYear_health2018.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\margDamages_EASIUR\Generation-MARREG-DAMEASIUR-egrid-byYear_health2018.csv


In [21]:
# Marginal damages [$/kWh]
# Inflate from 2010 to 2021
# Note only 2018 data available, used in place of 2021
df_margDamages_EASIUR_health = pd.DataFrame({
    'subregion_eGRID': df_margDamages_health2018['region'],
    'pollutant': df_margDamages_health2018['pollutant'],
    'unit': '[$/kWh]',
    '2030_decarb': '65% from 2021',
    'margDamages_dollarPerkWh_adjustVSL_ref': (df_margDamages_health2018['factor'] * (vsl_adjustment_factor) * (1/1000)) * (cpi_ratio_2021_2010),
    'margDamages_dollarPerkWh_adjustVSL_2018': (df_margDamages_health2018['factor'] * (vsl_adjustment_factor) * (1/1000)) * (cpi_ratio_2021_2010)
})
df_margDamages_EASIUR_health['margDamages_decarb_2030'] = df_margDamages_EASIUR_health['margDamages_dollarPerkWh_adjustVSL_ref'] - (df_margDamages_EASIUR_health['margDamages_dollarPerkWh_adjustVSL_ref'] * 0.65)
df_margDamages_EASIUR_health['reduction_margDamages_2030'] = df_margDamages_EASIUR_health['margDamages_dollarPerkWh_adjustVSL_2018'] - df_margDamages_EASIUR_health['margDamages_decarb_2030']
df_margDamages_EASIUR_health['reduction_margDamages_annual'] = df_margDamages_EASIUR_health['reduction_margDamages_2030'] / 9
# df_margDamages_EASIUR_health

In [22]:
# Combine them top to bottom
df_margDamages_EASIUR = pd.concat([df_margDamages_EASIUR_climate, df_margDamages_EASIUR_health], ignore_index=True)
df_margDamages_EASIUR

,subregion_eGRID,pollutant,unit,2030_decarb,margDamages_dollarPerkWh_adjustVSL_ref,margDamages_dollarPerkWh_adjustVSL_2018,margDamages_decarb_2030,reduction_margDamages_2030,reduction_margDamages_annual
0,AZNM,co2,[$/kWh],68% from 2005,0.093359,0.107904,0.029875,0.078029,0.006502
1,CAMX,co2,[$/kWh],68% from 2005,0.088012,0.088766,0.028164,0.060603,0.005050
2,ERCT,co2,[$/kWh],68% from 2005,0.106521,0.110424,0.034087,0.076338,0.006361
3,FRCC,co2,[$/kWh],68% from 2005,0.122589,0.093406,0.039228,0.054177,0.004515
4,MROE,co2,[$/kWh],68% from 2005,0.158571,0.158156,0.050743,0.107413,0.008951
...,...,...,...,...,...,...,...,...,...
83,SRTV,nox,[$/kWh],65% from 2021,0.002472,0.002472,0.000865,0.001607,0.000179
84,SRTV,pm25,[$/kWh],65% from 2021,0.006606,0.006606,0.002312,0.004294,0.000477
85,SRVC,so2,[$/kWh],65% from 2021,0.007316,0.007316,0.002560,0.004755,0.000528
86,SRVC,nox,[$/kWh],65% from 2021,0.002162,0.002162,0.000757,0.001405,0.000156


In [23]:
# Marginal Damages for a Gradually Decarbonizing Grid
df_margDamages_gridDecarb = df_margDamages_EASIUR.copy()

years = list(range(2019, 2051))

# Apply reductions
for year in years:
    column_name = f'margDamages_dollarPerkWh_adjustVSL_{year}'
    df_margDamages_gridDecarb[column_name] = df_margDamages_gridDecarb['margDamages_dollarPerkWh_adjustVSL_ref']  # Initialize

    for index, row in df_margDamages_gridDecarb.iterrows():  # Correctly unpack the index and row
        if year <= 2030:
            # Climate reduction (C02) applicable from 2019 to 2030
            # No Health reductions before 2022
            if 2019 <= year < 2022:
                if row['pollutant'] == 'co2':
                    df_margDamages_gridDecarb.at[index, column_name] = df_margDamages_gridDecarb.at[index, f'margDamages_dollarPerkWh_adjustVSL_{year-1}'] - df_margDamages_gridDecarb.at[index, 'reduction_margDamages_annual']
                else:
                    df_margDamages_gridDecarb.at[index, column_name] = df_margDamages_gridDecarb.at[index, f'margDamages_dollarPerkWh_adjustVSL_{year-1}']
            
            # Health reduction applicable from 2022 to 2030
            # Climate reductions continue
            elif year >= 2022:
                df_margDamages_gridDecarb.at[index, column_name] = df_margDamages_gridDecarb.at[index, f'margDamages_dollarPerkWh_adjustVSL_{year-1}'] - df_margDamages_gridDecarb.at[index, 'reduction_margDamages_annual']

        # Post-2030, damage values should be at the 2030 level
        else:
            df_margDamages_gridDecarb.at[index, column_name] = df_margDamages_gridDecarb.at[index, f'margDamages_dollarPerkWh_adjustVSL_2030']

df_margDamages_gridDecarb

,subregion_eGRID,pollutant,unit,2030_decarb,margDamages_dollarPerkWh_adjustVSL_ref,margDamages_dollarPerkWh_adjustVSL_2018,margDamages_decarb_2030,reduction_margDamages_2030,reduction_margDamages_annual,margDamages_dollarPerkWh_adjustVSL_2019,...,margDamages_dollarPerkWh_adjustVSL_2041,margDamages_dollarPerkWh_adjustVSL_2042,margDamages_dollarPerkWh_adjustVSL_2043,margDamages_dollarPerkWh_adjustVSL_2044,margDamages_dollarPerkWh_adjustVSL_2045,margDamages_dollarPerkWh_adjustVSL_2046,margDamages_dollarPerkWh_adjustVSL_2047,margDamages_dollarPerkWh_adjustVSL_2048,margDamages_dollarPerkWh_adjustVSL_2049,margDamages_dollarPerkWh_adjustVSL_2050
0,AZNM,co2,[$/kWh],68% from 2005,0.093359,0.107904,0.029875,0.078029,0.006502,0.101401,...,0.029875,0.029875,0.029875,0.029875,0.029875,0.029875,0.029875,0.029875,0.029875,0.029875
1,CAMX,co2,[$/kWh],68% from 2005,0.088012,0.088766,0.028164,0.060603,0.005050,0.083716,...,0.028164,0.028164,0.028164,0.028164,0.028164,0.028164,0.028164,0.028164,0.028164,0.028164
2,ERCT,co2,[$/kWh],68% from 2005,0.106521,0.110424,0.034087,0.076338,0.006361,0.104063,...,0.034087,0.034087,0.034087,0.034087,0.034087,0.034087,0.034087,0.034087,0.034087,0.034087
3,FRCC,co2,[$/kWh],68% from 2005,0.122589,0.093406,0.039228,0.054177,0.004515,0.088891,...,0.039228,0.039228,0.039228,0.039228,0.039228,0.039228,0.039228,0.039228,0.039228,0.039228
4,MROE,co2,[$/kWh],68% from 2005,0.158571,0.158156,0.050743,0.107413,0.008951,0.149204,...,0.050743,0.050743,0.050743,0.050743,0.050743,0.050743,0.050743,0.050743,0.050743,0.050743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,SRTV,nox,[$/kWh],65% from 2021,0.002472,0.002472,0.000865,0.001607,0.000179,0.002472,...,0.000865,0.000865,0.000865,0.000865,0.000865,0.000865,0.000865,0.000865,0.000865,0.000865
84,SRTV,pm25,[$/kWh],65% from 2021,0.006606,0.006606,0.002312,0.004294,0.000477,0.006606,...,0.002312,0.002312,0.002312,0.002312,0.002312,0.002312,0.002312,0.002312,0.002312,0.002312
85,SRVC,so2,[$/kWh],65% from 2021,0.007316,0.007316,0.002560,0.004755,0.000528,0.007316,...,0.002560,0.002560,0.002560,0.002560,0.002560,0.002560,0.002560,0.002560,0.002560,0.002560
86,SRVC,nox,[$/kWh],65% from 2021,0.002162,0.002162,0.000757,0.001405,0.000156,0.002162,...,0.000757,0.000757,0.000757,0.000757,0.000757,0.000757,0.000757,0.000757,0.000757,0.000757


In [24]:
# Create an empty dictionary to store the lookup data
dict_margDamages_gridDecarb = {}

for year in years:
    # Create an empty dictionary for the current year
    year_lookup = {}
    
    for _, row in df_margDamages_gridDecarb.iterrows():
        year_lookup[(row['subregion_eGRID'], row['pollutant'])] = row[f'margDamages_dollarPerkWh_adjustVSL_{str(year)}']
    
    # Add the year-specific lookup to the main lookup_data dictionary
    dict_margDamages_gridDecarb[year] = year_lookup

# Now, you have a lookup_data dictionary containing emissions factors for each state and year
dict_margDamages_gridDecarb

{2019: {('AZNM', 'co2'): 0.10140113554892047,
  ('CAMX', 'co2'): 0.08371628166873829,
  ('ERCT', 'co2'): 0.10406285163065675,
  ('FRCC', 'co2'): 0.08889105567336082,
  ('MROE', 'co2'): 0.14920444426441548,
  ('MROW', 'co2'): 0.14842635622351424,
  ('NEWE', 'co2'): 0.08180333245678206,
  ('NWPP', 'co2'): 0.129662604747659,
  ('NYCW', 'co2'): 0.08055916500106658,
  ('NYLI', 'co2'): 0.10338846929994001,
  ('NYUP', 'co2'): 0.08115292164008463,
  ('RFCE', 'co2'): 0.09933715524797929,
  ('RFCM', 'co2'): 0.12373499231743167,
  ('RFCW', 'co2'): 0.12982803965521555,
  ('RMPA', 'co2'): 0.1231835790161593,
  ('SPNO', 'co2'): 0.13645288226626034,
  ('SPSO', 'co2'): 0.11873459606535809,
  ('SRMV', 'co2'): 0.10540715212899736,
  ('SRMW', 'co2'): 0.14122130511570125,
  ('SRSO', 'co2'): 0.11514028525910261,
  ('SRTV', 'co2'): 0.11815691807773179,
  ('SRVC', 'co2'): 0.1196891414144477,
  ('AZNM', 'so2'): 0.001565519163910798,
  ('AZNM', 'nox'): 0.000511358363683762,
  ('AZNM', 'pm25'): 0.00180169834374

In [25]:
print("""
-------------------------------------------------------------------------------------------------------
Calculate Emissions Factors: FOSSIL FUELS
-------------------------------------------------------------------------------------------------------
""")

# Create a lookup dictionary for the national emissions factors
national_factors = df_margEmis_factors[df_margEmis_factors['state'] == 'National']
national_lookup = {(row['fuel_type'], row['pollutant']): row['margEmis_factor_adjusted'] for _, row in national_factors.iterrows()}

# Create a lookup dictionary for the state-specific emissions factors for electricity
electricity_factors = df_margEmis_factors[df_margEmis_factors['fuel_type'] == 'electricity']
electricity_lookup = {(row['pollutant'], row['state']): row['margEmis_factor_adjusted'] for _, row in electricity_factors.iterrows()}

pollutants = ['so2', 'nox', 'pm25', 'co2']

# ELECTRICITY CEDM DAMAGES LOOKUP
damages_CEDM_lookup = {(row['pollutant'], row['subregion_eGRID']): row['margDamages_dollarPerkWh_adjustVSL_ref'] for _, row in df_margDamages_EASIUR.iterrows()}

# FOSSIL FUELS DAMAGES LOOKUP
# Create a damages_fossilFuel_lookup dictionary from df_margSocialCosts_EASIUR
damages_fossilFuel_lookup = df_margSocialCosts_EASIUR.groupby(['Longitude', 'Latitude']).first().to_dict()


-------------------------------------------------------------------------------------------------------
Calculate Emissions Factors: FOSSIL FUELS
-------------------------------------------------------------------------------------------------------



### Step 5: Calculate End-use specific marginal damages
**I used the total emissions column for each of the end uses for the following reasons:**
- Most homes only have 1 of each end-use, so it is unlikely that the homes have a significant consumption values from different fuel types. Thus, the total consumption and total emissions column (sum of each dwelling units consumption by end-use for each fuel) is fine to use to calculate marginal damages (social cost)
- We can visualize the emissions in 2 by 2 grid (CO2, PM25, SO2, NOx) with each appliance's heating fuel in a different shape or color. 

### Baseline Marginal Damages: WHOLE-HOME

In [26]:
print("""
-------------------------------------------------------------------------------------------------------
Step 5: Calculate End-use specific marginal damages
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Baseline Marginal Damages: WHOLE-HOME
-------------------------------------------------------------------------------------------------------
""")

# calculate_marginal_damages(df, grid_decarb=False)
df_euss_am_baseline_home = calculate_marginal_damages(df=df_euss_am_baseline_home,
                                                      grid_decarb=False
                                                     )
df_euss_am_baseline_home


-------------------------------------------------------------------------------------------------------
Step 5: Calculate End-use specific marginal damages
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Baseline Marginal Damages: WHOLE-HOME
-------------------------------------------------------------------------------------------------------

End-use category: heating
Baseline Damages for heating - so2:
410884    1.69
411011    0.92
411058    0.80
411079    1.03
411192    0.90
Name: baseline_heating_damages_so2, dtype: float64
Baseline Damages for heating - nox:
410884    114.32
411011     61.89
411058     53.90
411079     69.64
411192     60.81
Name: baseline_heating_damages_nox, dtype: float64
Baseline Damages for heating - pm25:
410884    54.39
411011    29.45
411058    25.65
411079    33.13
411192    28.93
Name: ba

,bldg_id,square_footage,census_region,building_america_climate_zone,cambium_GEA_region,state,city,county,puma,county_and_puma,...,baseline_clothesDrying_damages_pm25,baseline_clothesDrying_damages_co2,baseline_clothesDrying_damages_health,baseline_clothesDrying_damages_climate,baseline_cooking_damages_so2,baseline_cooking_damages_nox,baseline_cooking_damages_pm25,baseline_cooking_damages_co2,baseline_cooking_damages_health,baseline_cooking_damages_climate
410884,1081,1690.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001702,"G4200030, G42001702",...,9.40,49.01,19.08,49.01,0.63,1.82,1.49,39.75,3.94,39.75
411011,4075,2152.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001701,"G4200030, G42001701",...,1.23,23.68,2.99,23.68,8.40,1.03,9.16,47.71,18.59,47.71
411058,5187,1220.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001702,"G4200030, G42001702",...,9.40,49.01,19.08,49.01,0.63,1.82,1.49,39.75,3.94,39.75
411079,5617,1690.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001701,"G4200030, G42001701",...,9.40,49.01,19.08,49.01,0.63,1.82,1.49,39.75,3.94,39.75
411192,7867,1220.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001702,"G4200030, G42001702",...,7.52,39.21,15.26,39.21,7.00,0.85,7.63,39.74,15.48,39.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433429,538404,8194.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001701,"G4200030, G42001701",...,2.56,49.48,6.23,49.48,13.01,1.59,14.18,73.89,28.78,73.89
433435,538501,1220.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001702,"G4200030, G42001702",...,0.91,17.64,2.22,17.64,0.52,1.52,1.24,33.19,3.28,33.19
433547,541686,1690.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001702,"G4200030, G42001702",...,11.29,58.81,22.90,58.81,0.75,2.18,1.78,47.69,4.71,47.69
433583,542571,1220.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001701,"G4200030, G42001701",...,0.91,17.64,2.22,17.64,0.52,1.52,1.24,33.19,3.28,33.19


## Private Perspective: Annual Energy Costs

### Step 1: Obtain Level Energy Fuel Cost Data from the EIA
**Data Sources for Excel workbook containing state average Residential fuel cost for each fuel in 2018**
- EIA State Electricity Price: https://www.eia.gov/electricity/state/archive/2018/
- EIA Natural Gas Prices: https://www.eia.gov/dnav/ng/ng_pri_sum_dcu_SPA_a.htm
- Propane and Fuel Oil: EIA March 2023 Short Term Energy Outlook
    - https://www.eia.gov/outlooks/steo/pdf/wf01.pdf
    - Table WF01: Average Consumer Prices and Expenditures for Heating Fuels During the Winter
    - US Average: 2018-2019 Data

In [27]:
print("""
-------------------------------------------------------------------------------------------------------
Private Perspective: Annual Energy Costs
-------------------------------------------------------------------------------------------------------
- Step 1: Obtain Level Energy Fuel Cost Data from the EIA
- Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Step 1: Obtain Level Energy Fuel Cost Data from the EIA
-------------------------------------------------------------------------------------------------------
**Data Sources for Excel workbook containing state average Residential fuel cost for each fuel in 2018**
- EIA State Electricity Price: https://www.eia.gov/electricity/state/archive/2018/
- EIA Natural Gas Prices: https://www.eia.gov/dnav/ng/ng_pri_sum_dcu_SPA_a.htm
- Propane and Fuel Oil: EIA March 2023 Short Term Energy Outlook
    - https://www.eia.gov/outlooks/steo/pdf/wf01.pdf
    - Table WF01: Average Consumer Prices and Expenditures for Heating Fuels During the Winter
    - US Average: 2018-2019 Data
-------------------------------------------------------------------------------------------------------
""")

filename = 'fuel_prices.xlsx'
relative_path = os.path.join(r"fuel_prices", filename)
file_path = os.path.join(project_root, relative_path)
df_fuelPrices_perkWh = pd.read_excel(file_path, sheet_name='fuel_prices')

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# Create a new column called "cost_per_kWh"
df_fuelPrices_perkWh['cost_per_kWh'] = 0.0

# Convert to $/kWh equivalent
# https://www.eia.gov/energyexplained/units-and-calculators/british-thermal-units.php
for index, row in df_fuelPrices_perkWh.iterrows():
    
    # Propane: (dollars per gallon) * (1 gallon propane/91,452 BTU) * (3412 BTU/1 kWh)
    if row['fuel_type'] == 'propane':
        df_fuelPrices_perkWh.at[index, 'cost_per_kWh'] = row['cost_per_unit'] * (1/91452) * (3412/1)
    
    # Fuel Oil: (dollars/gallon) * (1 gallon heating oil/138,500 BTU) * (3412 BTU/1 kWh)
    elif row['fuel_type'] == 'fuelOil':
        df_fuelPrices_perkWh.at[index, 'cost_per_kWh'] = row['cost_per_unit'] * (1/138500) * (3412/1)
    
    # Natural Gas: (dollars/cf) * (thousand cf/1000 cf) * (1 cf natural gas/1039 BTU) * (3412 BTU/1 kWh)
    elif row['fuel_type'] == 'naturalGas':
        df_fuelPrices_perkWh.at[index, 'cost_per_kWh'] = row['cost_per_unit'] * (1/1000) * (1/1039) * (3412/1)
    
    # Electricity: convert cents per kWh to $ per kWh
    elif row['fuel_type'] == 'electricity':
        df_fuelPrices_perkWh.at[index, 'cost_per_kWh'] = row['cost_per_unit'] / 100

df_fuelPrices_perkWh


-------------------------------------------------------------------------------------------------------
Private Perspective: Annual Energy Costs
-------------------------------------------------------------------------------------------------------
- Step 1: Obtain Level Energy Fuel Cost Data from the EIA
- Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Step 1: Obtain Level Energy Fuel Cost Data from the EIA
-------------------------------------------------------------------------------------------------------
**Data Sources for Excel workbook containing state average Residential fuel cost for each fuel in 2018**
- EIA State Electricity Price: https://www.eia.gov/electricity/state/archive/2018/
- EIA Natural Gas Prices: https://www.eia.gov/dnav/ng/ng_pri_sum_dcu_SPA_a.htm


,year,fuel_type,state_region,cost_per_unit,units,cost_per_kWh
0,2018,propane,Northeast,3.19,dollars per gallon,0.119016
1,2018,propane,Midwest,1.72,dollars per gallon,0.064172
2,2018,propane,South,2.63,dollars per gallon,0.098123
3,2018,fuelOil,National,3.07,dollars per gallon,0.075631
4,2018,naturalGas,National,10.50,dollars per cf,0.034481
...,...,...,...,...,...,...
103,2018,electricity,VT,18.02,cents per kWh,0.180200
104,2018,electricity,WA,9.75,cents per kWh,0.097500
105,2018,electricity,WI,14.02,cents per kWh,0.140200
106,2018,electricity,WV,11.18,cents per kWh,0.111800


### Step 2: Calculate Annual Operating (Fuel) Costs

### Baseline Fuel Cost: WHOLE-HOME

In [28]:
print("""
-------------------------------------------------------------------------------------------------------
Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
- Create a mapping dictionary for fuel types
- Create new merge columns to ensure a proper match.
- Merge df_copy with df_fuel_prices to get fuel prices for electricity, natural gas, propane, and fuel oil
- Calculate the per kWh fuel costs for each fuel type and region
- Calculate the baseline fuel cost 
- Add fuel oil cost for non-cooking and non-clothesDrying categories
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Baseline Fuel Cost: WHOLE-HOME
-------------------------------------------------------------------------------------------------------
""")

# df_euss_am_baseline_home = df_euss_am_baseline_home.copy()
# calculate_annual_fuelCost(df, category, state_region, df_fuelPrices_perkWh, cpi_ratio)
df_euss_am_baseline_home = calculate_annual_fuelCost(df=df_euss_am_baseline_home,
                                                        state_region=input_state,
                                                        df_fuelPrices_perkWh=df_fuelPrices_perkWh,
                                                        cpi_ratio=cpi_ratio_2021_2018                                                        
                                                       )
df_euss_am_baseline_home


-------------------------------------------------------------------------------------------------------
Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
- Create a mapping dictionary for fuel types
- Create new merge columns to ensure a proper match.
- Merge df_copy with df_fuel_prices to get fuel prices for electricity, natural gas, propane, and fuel oil
- Calculate the per kWh fuel costs for each fuel type and region
- Calculate the baseline fuel cost 
- Add fuel oil cost for non-cooking and non-clothesDrying categories
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Baseline Fuel Cost: WHOLE-HOME
-------------------------------------------------------------------------------------------------------



,bldg_id,square_footage,census_region,building_america_climate_zone,cambium_GEA_region,state,city,county,puma,county_and_puma,...,fuel_type_clothesDrying,fuel_type_cooking,fuelPrice_electricity_perkWh,fuelPrice_naturalGas_perkWh,fuelPrice_propane_perkWh,fuelPrice_fuelOil_perkWh,baseline_heating_fuelCost,baseline_waterHeating_fuelCost,baseline_clothesDrying_fuelCost,baseline_cooking_fuelCost
410884,1081,1690.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001702,"G4200030, G42001702",...,electricity,naturalGas,0.149887,0.039867,0.128431,0.081613,2124.17,155.36,59.96,33.54
411011,4075,2152.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001701,"G4200030, G42001701",...,naturalGas,electricity,0.149887,0.039867,0.128431,0.081613,1150.01,147.96,18.83,58.38
411058,5187,1220.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001702,"G4200030, G42001702",...,electricity,naturalGas,0.149887,0.039867,0.128431,0.081613,1001.62,160.31,59.96,33.54
411079,5617,1690.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001701,"G4200030, G42001701",...,electricity,naturalGas,0.149887,0.039867,0.128431,0.081613,1294.01,167.49,59.96,33.54
411192,7867,1220.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001702,"G4200030, G42001702",...,electricity,electricity,0.149887,0.039867,0.128431,0.081613,1129.86,210.19,47.97,48.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433429,538404,8194.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001701,"G4200030, G42001701",...,naturalGas,electricity,0.149887,0.039867,0.128431,0.081613,4585.79,292.67,39.36,90.40
433435,538501,1220.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001702,"G4200030, G42001702",...,naturalGas,naturalGas,0.149887,0.039867,0.128431,0.081613,902.05,111.17,14.03,28.02
433547,541686,1690.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001702,"G4200030, G42001702",...,electricity,naturalGas,0.149887,0.039867,0.128431,0.081613,898.41,259.58,71.95,40.25
433583,542571,1220.0,Northeast,Cold,RFCWc,PA,Pittsburgh,G4200030,G42001701,"G4200030, G42001701",...,naturalGas,naturalGas,0.149887,0.039867,0.128431,0.081613,422.65,170.44,14.03,28.02


### Area Median Income Data Used to determine LMI Designation and IRA Rebates Eligibility/Amount

In [29]:
# Collect Area Median Income Data at PUMA-resolution
filename = "nhgis0002_ds239_20185_puma.csv"
relative_path = os.path.join(r"equity_data", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_county_medianIncome = pd.read_csv(file_path, encoding='ISO-8859-1')
df_county_medianIncome = df_county_medianIncome.drop(0)
df_county_medianIncome = df_county_medianIncome.reset_index(drop=True)

cols_interest = ['GISJOIN', 'STUSAB', 'STATE', 'PUMAA', 'GEOID', 'NAME_E', 'AJZAE001', 'AJZAM001']
df_county_medianIncome = df_county_medianIncome[cols_interest]
df_county_medianIncome = df_county_medianIncome.rename(columns={"GISJOIN": "gis_joinID_puma", "STUSAB": "state_abbrev", "STATE": "state", "PUMAA": "puma_code", "GEOID": "census_geoID", "NAME_E": "name_estimate", "AJZAE001": "median_income_USD2018", "AJZAM001": "median_income_USD2018_marginOfError"})
df_county_medianIncome

Retrieved data for filename: nhgis0002_ds239_20185_puma.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\equity_data\nhgis0002_ds239_20185_puma.csv




,gis_joinID_puma,state_abbrev,state,puma_code,census_geoID,name_estimate,median_income_USD2018,median_income_USD2018_marginOfError
0,G01000100,AL,Alabama,100,79500US0100100,"Lauderdale, Colbert, Franklin & Marion (Northe...",44779,1317
1,G01000200,AL,Alabama,200,79500US0100200,Limestone & Madison (Outer) Counties--Huntsvil...,64779,2670
2,G01000301,AL,Alabama,301,79500US0100301,Huntsville (North) & Madison (East) Cities PUM...,62399,1595
3,G01000302,AL,Alabama,302,79500US0100302,"Huntsville City (Central & South) PUMA, Alabama",52811,1814
4,G01000400,AL,Alabama,400,79500US0100400,"DeKalb & Jackson Counties PUMA, Alabama",39499,1447
...,...,...,...,...,...,...,...,...
2373,G72000902,PR,Puerto Rico,902,79500US7200902,"Trujillo Alto, Canóvanas, Loíza & Carolina (Su...",25262,972
2374,G72001001,PR,Puerto Rico,1001,79500US7201001,"Caguas Municipio PUMA, Puerto Rico",25006,1128
2375,G72001002,PR,Puerto Rico,1002,79500US7201002,"Gurabo, San Lorenzo, Juncos & Las Piedras Muni...",22391,888
2376,G72001101,PR,Puerto Rico,1101,79500US7201101,"Río Grande, Fajardo, Luquillo, Ceiba, Vieques ...",20621,815


# Model Runtime

In [30]:
# Get the current datetime again
end_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Calculate the elapsed time
elapsed_time = datetime.strptime(end_time, "%Y-%m-%d_%H-%M-%S") - datetime.strptime(start_time, "%Y-%m-%d_%H-%M-%S")

# Format the elapsed time
elapsed_seconds = elapsed_time.total_seconds()
elapsed_minutes = int(elapsed_seconds // 60)
elapsed_seconds = int(elapsed_seconds % 60)

# Print the elapsed time
print(f"The code took {elapsed_minutes} minutes and {elapsed_seconds} seconds to execute.")

The code took 0 minutes and 30 seconds to execute.
